# Training

Data format:

| location_x | location_y | AP1_rssi | AP2_rssi | AP3_rssi |
|------------|------------|----------|----------|----------|
|relative x location|relative y location|AP1 RSSI|AP2 RSSI|AP3 RSSI|


We will have a cmongodb collectrion per geometric form we are using for training

The training will be done in 2 fases, one where we combine all the 

In [12]:
import torch
from torch.utils.data import Dataset
from pymongo import MongoClient
import numpy as np


    
def process_data(data):
    """
    Preprocess the MongoDB documents into a single array with 5 columns.
    Columns: AP1_rssi, AP2_rssi, AP3_rssi, location_x, location_y
    
    Handles NaN values by:
    1. Replacing NaN RSSI values with -100 (standard for missing signal)
    2. Ensuring coordinates are always valid numbers
    """
    combined_data = []
    
    for entry in data:
        # Safely extract RSSI values, handling missing/NaN values
        rssi_values = [
            float(entry.get('AP1_rssi', -100)) if entry.get('AP1_rssi', -100) != None else -100,
            float(entry.get('AP2_rssi', -100)) if entry.get('AP2_rssi', -100) != None else -100,
            float(entry.get('AP3_rssi', -100)) if entry.get('AP3_rssi', -100) != None else -100
        ]
        
        # Validate coordinates
        try:
            x_coord = float(entry['location_x'])
            y_coord = float(entry['location_y'])
            if np.isnan(x_coord) or np.isnan(y_coord):
                continue  # Skip this entry if coordinates are invalid
        except (KeyError, ValueError):
            continue  # Skip this entry if coordinates are missing or invalid
            
        # Combine all values into one row
        combined_row = rssi_values + [x_coord, y_coord]
        combined_data.append(combined_row)
    
    # Convert to numpy array and verify no NaNs remain
    result = np.array(combined_data, dtype=np.float32)
    assert not np.isnan(result).any(), "NaN values detected in final output!"
    
    return result

def get_dataset(collection_name, db_name):
    """
    Args:
        collection_name (str): Name of the MongoDB collection to use
        db_name (str): Name of the MongoDB database
    """
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:28910/')
    db = client[db_name]
    collection = db[collection_name]
    
    # Load all data from the collection
    data = list(collection.find())
    
    # Preprocess the data to extract features and labels
    return process_data(data)


def split_combined_data(combined_array, num_ap=3):

    # Split the array into features (RSSI values) and labels (coordinates)
    features = combined_array[:, :num_ap]  # First num_ap columns are RSSI values
    labels = combined_array[:, num_ap:]    # Last 2 columns are coordinates
    
    return features, labels

def combine_arrays(arrays):
    return np.vstack(arrays)

def shuffle_array(arr, random_state=None):
    np.random.seed(random_state)
    shuffled_arr = arr.copy()
    np.random.shuffle(shuffled_arr)
    return shuffled_arr



# Training

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WiFiPositionModel(nn.Module):
    def __init__(self, input_size=3, output_size=2):
        super(WiFiPositionModel, self).__init__()
        
        # Feature extraction branch
        self.feature_net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.4),
            
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.4),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1)
        )
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 256),
            nn.Softmax(dim=1)
        )
        
        # Position prediction branch
        self.position_net = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.1),
            
            nn.Linear(64, output_size)
        )
        
        # Uncertainty estimation
        self.uncertainty_net = nn.Sequential(
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, output_size),
            nn.Softplus()
        )
        
    def forward(self, x):
        # Feature extraction
        features = self.feature_net(x)
        
        # Attention mechanism
        attention_weights = self.attention(features)
        attended_features = features * attention_weights
        
        # Position prediction
        position = self.position_net(attended_features)
        
        # Uncertainty estimation
        uncertainty = self.uncertainty_net(attended_features)
        
        return position, uncertainty

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_wifi_model(X_train, y_train, X_val, y_val, epochs=100, batch_size=32, learning_rate=0.001):
    """
    Train a neural network model for WiFi positioning and visualize results.
    
    Args:
        X_train (np.array): Training features (RSSI values)
        y_train (np.array): Training labels (coordinates)
        X_val (np.array): Validation features
        y_val (np.array): Validation labels
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
        learning_rate (float): Learning rate for optimizer
        
    Returns:
        tuple: (trained model, training history dictionary)
    """


    print(f"Using device: {device}")
    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    print(f"Training tensor is on: {X_train_tensor.device}")
    
    # Create DataLoader for batch training
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model, loss function, and optimizer
    model = WiFiPositionModel(input_size=X_train.shape[1]).to(device)
    print(f"Model is on: {next(model.parameters()).device}")
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    train_loss_history = []
    val_loss_history = []
    
    for epoch in range(epochs):
        model.train()
        batch_losses = []
        
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs, _ = model(inputs)  # Only use position output
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
        
        # Calculate epoch training loss
        train_loss = np.mean(batch_losses)
        train_loss_history.append(train_loss)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs, _ = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor).item()
            val_loss_history.append(val_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Final evaluation
    model.eval()
    with torch.no_grad():
        # Training set evaluation
        train_preds, _ = model(X_train_tensor)
        train_preds = train_preds.cpu().numpy()  # Move back to CPU for sklearn
        train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
        train_mae = mean_absolute_error(y_train, train_preds)
        
        # Validation set evaluation
        val_preds, _ = model(X_val_tensor)
        val_preds = val_preds.cpu().numpy()
        val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        val_mae = mean_absolute_error(y_val, val_preds)
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_loss_history, label='Training Loss')
    plt.plot(val_loss_history, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot actual vs predicted for validation set
    plt.subplot(1, 2, 2)
    plt.scatter(y_val[:, 0], y_val[:, 1], label='Actual', alpha=0.5)
    plt.scatter(val_preds[:, 0], val_preds[:, 1], label='Predicted', alpha=0.5)
    plt.title('Actual vs Predicted Positions')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print("\nFinal Metrics:")
    print(f"Training RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}")
    print(f"Validation RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}")
    
    # Return model and history
    history = {
        'train_loss': train_loss_history,
        'val_loss': val_loss_history,
        'train_metrics': {'rmse': train_rmse, 'mae': train_mae},
        'val_metrics': {'rmse': val_rmse, 'mae': val_mae}
    }
    
    return model, history


In [ ]:
# Get datasets from all collections
datasets = [
    get_dataset("wifi_data_reto_grande", "wifi_data_db"),
    get_dataset("wifi_data_reto_pequeno", "wifi_data_db"),
    get_dataset("wifi_data_reto_medio", "wifi_data_db")
]

# Combine all datasets into one array
combined_data = combine_arrays(datasets)

# Shuffle the combined data
shuffled_data = shuffle_array(combined_data)

# Split into features and labels
#training_x, training_y = split_combined_data(shuffled_data)
#validation_x, validation_y = split_combined_data(get_dataset("wifi_data_reto_medio", "wifi_data_db"))
global_array_x, global_array_y = split_combined_data(shuffled_data)


# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(global_array_x, global_array_y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = training_x, validation_x, training_y, validation_y

train_wifi_model(X_train, y_train, X_val, y_val,epochs=2000)

Using device: cuda


UnboundLocalError: cannot access local variable 'model' where it is not associated with a value